In [15]:
import torch
from torch import nn
from torchinfo import summary

from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier

import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from tqdm.auto import tqdm
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
import numpy  as np
import tracemalloc 

import json
import os
import glob
import pickle
from itertools import combinations
import gc
import time
import random

import warnings
warnings.filterwarnings("ignore")

In [2]:
TEST = True

In [3]:
DATA_ROOT = "../data/compgan_dataset/"

train_data_file_name_ = "train_data{}.npy"
train_label_file_name_ = "train_label{}.npy"
test_data_file_name_ = "test_data{}.npy"
test_label_file_name_ = "test_label{}.npy"

TRAIN_FOLDER_PATH = os.path.join(DATA_ROOT, "train")
TEST_FOLDER_PATH = os.path.join(DATA_ROOT, "test")
RESULT_FOLDER_PATH = os.path.join(DATA_ROOT, "results")
PREV_CHECKPOINT_FOLDER_PATH = os.path.join(DATA_ROOT, "prev_checkpoint")

assert os.path.isdir(TRAIN_FOLDER_PATH) and os.path.isdir(TEST_FOLDER_PATH)
os.makedirs(RESULT_FOLDER_PATH, exist_ok=True)

data_files = sorted(glob.glob(os.path.join(TRAIN_FOLDER_PATH, train_data_file_name_.format("*"))))
print(data_files)
print(len(data_files))

['../data/compgan_dataset/train/train_data0.npy', '../data/compgan_dataset/train/train_data1.npy', '../data/compgan_dataset/train/train_data10.npy', '../data/compgan_dataset/train/train_data11.npy', '../data/compgan_dataset/train/train_data12.npy', '../data/compgan_dataset/train/train_data13.npy', '../data/compgan_dataset/train/train_data14.npy', '../data/compgan_dataset/train/train_data15.npy', '../data/compgan_dataset/train/train_data2.npy', '../data/compgan_dataset/train/train_data3.npy', '../data/compgan_dataset/train/train_data4.npy', '../data/compgan_dataset/train/train_data5.npy', '../data/compgan_dataset/train/train_data6.npy', '../data/compgan_dataset/train/train_data7.npy', '../data/compgan_dataset/train/train_data8.npy', '../data/compgan_dataset/train/train_data9.npy']
16


In [4]:
USER_NUM = 16
SENSOR_NUM = 7
EACH_SENSOR_CHANNEL = 6
assert USER_NUM == len(data_files)
feature_num = SENSOR_NUM * EACH_SENSOR_CHANNEL

In [5]:
# important

label_list = ['歩行(平地)',
 '歩行(階段)',
 'ベッド上での起き上がり',
 'ベッド椅子間の乗り移り(立つ)',
 'ベッド椅子間の乗り移り(立たない)',
 '立ち座り',
 '座位保持・座位バランス',
 '立位保持・立位バランス',
 '関節可動域増大訓練(肩)',
 '関節可動域増大訓練(股関節)']

label_dict = dict(enumerate(label_list))

In [6]:
# important
eng_label_dict = dict(zip(
    label_list,
    ['Walking', 'Upstair', 'Bed_Standup', 'Change_Bed', 'Change_Bed_Standup', 'Sit_Down', 'Sit', 'Stand', 'Shoulder_Exercise', 'Hip_Exercise']
))

eng_label_list = [eng_label_dict[i] for i in label_list]

In [7]:
# Update after 2023/11/11 houkoku
class CustomTrainDataset(Dataset):
    TRAIN_MODE = "train"
    TEST_MODE = "test"
    
    def __init__(self, mode, feature_data, label_data, missing_sensor_numbers=0):
        self.mode = mode
        assert mode in [self.TRAIN_MODE, self.TEST_MODE]
        
        self.features = feature_data
        self.label = label_data
        assert len(self.features) == len(self.label), "features len is not equal to label len"
        self.missing_sensor_numbers = missing_sensor_numbers

        self.missing_index_list = []
        for missing_count in range(missing_sensor_numbers + 1):
            for missing_index in combinations(range(SENSOR_NUM), missing_count):
                self.missing_index_list.append(missing_index)

    def transform(self, one_feature, missing_sensor_id_list):
        # Make one sensor data become 0
        one_feature_cp = one_feature.copy()
        
        for missing_sensor_id in missing_sensor_id_list:
            one_feature_cp[:, missing_sensor_id*6:(missing_sensor_id+1)*6] = 0
        return one_feature_cp
        
    def __len__(self):

        # take all available missing pattern * data number
        return len(self.features) * len(self.missing_index_list)
    
    def __getitem__(self, idx):
        # take all available missing pattern
        missing_sensor_id_list = self.missing_index_list[ idx // len(self.features) ]
        x = self.transform(self.features[ idx % len(self.features) ], missing_sensor_id_list)
        label = self.label[idx % len(self.features)]
        return x, int(label)


In [10]:
class DataModule(pl.LightningDataModule):
    STANDARDIZE = False
    
    def __init__(self, test_user, missing_sensor_numbers, batch_size=2048):
        super().__init__()
        self.test_user = test_user
        self.missing_sensor_numbers = missing_sensor_numbers
        self.batch_size = batch_size
        self.scaler = None

    def load_data(self, mode):
        if mode == "train":
            folder_path = TRAIN_FOLDER_PATH
            data_file_name = train_data_file_name_
            label_file_name = train_label_file_name_

            train_data_file_name = data_file_name.format(self.test_user)
            train_label_file_name = label_file_name.format(self.test_user)
    
            train_data_file_path = os.path.join(folder_path, train_data_file_name)
            train_label_file_path = os.path.join(folder_path, train_label_file_name)
            train_val_data, train_val_label = np.load(train_data_file_path), np.load(train_label_file_path)
            l, s, d, w = train_val_data.shape
            self.scaler = StandardScaler()
            # train_val_data = self.scaler.fit_transform(train_val_data.reshape(l, -1)).reshape(l, s, d, w)
            train_val_data = train_val_data.reshape(l, s ,d, w).transpose(0, 3, 1, 2)
            
        elif mode == "test":
            folder_path = TEST_FOLDER_PATH
            data_file_name = test_data_file_name_
            label_file_name = test_label_file_name_
    
            train_data_file_name = data_file_name.format(self.test_user)
            train_label_file_name = label_file_name.format(self.test_user)
    
            train_data_file_path = os.path.join(folder_path, train_data_file_name)
            train_label_file_path = os.path.join(folder_path, train_label_file_name)
            train_val_data, train_val_label = np.load(train_data_file_path), np.load(train_label_file_path)
            l, s, d, w = train_val_data.shape
            # train_val_data = self.scaler.transform(train_val_data.reshape(l, -1)).reshape(l, s, d, w)
            train_val_data = train_val_data.reshape(l, s ,d, w).transpose(0, 3, 1, 2)
    
        return train_val_data, train_val_label

    def setup(self, stage: str):
        # Assign Train/val split(s) for use in Dataloaders
        
        if stage == "validate" or stage == "fit":
            train_val_data, train_val_label = self.load_data("train")
            self.train_data, self.val_data, self.train_label, self.val_label = train_test_split(
                train_val_data, train_val_label, test_size=0.2, train_size=0.8, random_state=42, shuffle=True)

            self.train_dataset = CustomTrainDataset(
                CustomTrainDataset.TRAIN_MODE, self.train_data, self.train_label, missing_sensor_numbers=self.missing_sensor_numbers)
            self.val_dataset = CustomTrainDataset(
                CustomTrainDataset.TEST_MODE, self.val_data, self.val_label, missing_sensor_numbers=self.missing_sensor_numbers)

        elif stage == "test" or stage == "predict":
            train_val_data, train_val_label = self.load_data("test")
            self.test_data = train_val_data
            self.test_label = train_val_label

            self.test_dataset = CustomTrainDataset(
                CustomTrainDataset.TEST_MODE, self.test_data, self.test_label, missing_sensor_numbers=self.missing_sensor_numbers)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=4, shuffle=True, pin_memory=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size,  num_workers=4, shuffle=False, pin_memory=True)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size,  num_workers=4, shuffle=False, pin_memory=True)

    def predict_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size,  num_workers=4, shuffle=False, pin_memory=True)

    def teardown(self, stage):
        print("teardown")
        if stage == "validate" or stage == "fit":
            del self.train_data, self.train_label
            del self.val_data, self.val_label
            del self.train_dataset
            del self.val_dataset
            
        elif stage == "test" or stage == "predict":
            del self.test_data, self.test_label
            del self.test_dataset
        gc.collect()


In [22]:
all_test_pred = {}

for missing_sensor_numbers in [0]: ## Changed for 1 missing sensor

    all_user_y_true = []
    all_user_y_pred = []
    
    for user in tqdm(range(16)): ## Changed for user 2 only
        data_module = DataModule(test_user=user, missing_sensor_numbers=missing_sensor_numbers, batch_size=512)
        
        train_data, train_label = data_module.load_data("train")
        train_data_reshape = train_data.reshape(train_data.shape[0], -1)

        classifier = RandomForestClassifier(n_estimators=200, criterion="gini")
        classifier.fit(train_data_reshape, train_label)

        test_data, test_label = data_module.load_data("test")
        test_data_reshape = test_data.reshape(test_data.shape[0], -1)

        test_pred = classifier.predict(test_data_reshape)
        
        all_user_y_true.extend(test_label)
        all_user_y_pred.extend(test_pred)
        
    all_test_pred[missing_sensor_numbers] = (all_user_y_true, all_user_y_pred)
        

100%|██████████| 16/16 [22:27<00:00, 84.20s/it]


In [23]:
all_user_y_true, all_user_y_pred = all_test_pred[0]

In [25]:
# Accuracy:
print(accuracy_score(all_user_y_true, all_user_y_pred))

0.33536121673003805


In [29]:
print(f1_score(all_user_y_true, all_user_y_pred, average="macro"))

0.32099072209201396
